In [1]:
import numpy as np

In [2]:
topic = {}
tfidf = dict(list(zip('cat dog apple lion NYC love'.split(),np.random.rand(6))))

In [3]:
topic['petness'] = (.3 * tfidf['cat'] +\
   .3 * tfidf['dog'] +\
    0 * tfidf['apple'] +\
    0 * tfidf['lion'] -\
    .2 * tfidf['NYC'] +\
     .2 * tfidf['love'])

In [4]:
topic['animalness'] = (.1 * tfidf['cat'] +\
    .1 * tfidf['dog'] -\
    .1 * tfidf['apple'] +\
    .5 * tfidf['lion'] +\
    .1 * tfidf['NYC'] -\
    1 * tfidf['love'])

In [5]:
topic['cityness'] = ( 0 * tfidf['cat'] -\
    .1 * tfidf['dog'] +\
    .2 * tfidf['apple'] -\
    .1 * tfidf['lion'] +\
    .5 * tfidf['NYC'] +\
    .1 * tfidf['love'])

In [6]:
word_vector = {}
word_vector['cat'] = .3*topic['petness'] +.1*topic['animalness'] +0*topic['cityness']
word_vector['dog'] = .3*topic['petness'] +.1*topic['animalness'] -.1*topic['cityness']
word_vector['apple']= 0*topic['petness'] -.1*topic['animalness'] +.2*topic['cityness']
word_vector['lion'] = 0*topic['petness'] +.5*topic['animalness'] -.1*topic['cityness']
word_vector['NYC'] = -.2*topic['petness'] +.1*topic['animalness'] +.5*topic['cityness']
word_vector['love'] = .2*topic['petness'] -.1*topic['animalness'] +.1*topic['cityness']

In [7]:
import pandas as pd
from nlpia.data.loaders import get_data
pd.options.display.width = 120
sms = get_data('sms-spam')
index = ['sms{}{}'.format(i, '!'*j) for (i,j) in zip(range(len(sms)), sms.spam)]
sms = pd.DataFrame(sms.values, columns=sms.columns, index=index)
sms['spam'] = sms.spam.astype(int)

INFO:nlpia.constants:Starting logger in nlpia.constants...
INFO:nlpia.loaders:No BIGDATA index found in C:\Anaconda\python\lib\site-packages\nlpia\data\bigdata_info.csv so copy C:\Anaconda\python\lib\site-packages\nlpia\data\bigdata_info.latest.csv to C:\Anaconda\python\lib\site-packages\nlpia\data\bigdata_info.csv if you want to "freeze" it.
INFO:nlpia.futil:Reading CSV with `read_csv(*('C:\\Anaconda\\python\\lib\\site-packages\\nlpia\\data\\mavis-batey-greetings.csv',), **{'low_memory': False})`...
INFO:nlpia.futil:Reading CSV with `read_csv(*('C:\\Anaconda\\python\\lib\\site-packages\\nlpia\\data\\sms-spam.csv',), **{'low_memory': False})`...
INFO:nlpia.futil:Reading CSV with `read_csv(*('C:\\Anaconda\\python\\lib\\site-packages\\nlpia\\data\\sms-spam.csv',), **{'nrows': None, 'low_memory': False})`...


In [8]:
sms.head(6)

,spam,text
sms0,0,"Go until jurong point, crazy.. Available only ..."
sms1,0,Ok lar... Joking wif u oni...
sms2!,1,Free entry in 2 a wkly comp to win FA Cup fina...
sms3,0,U dun say so early hor... U c already then say...
sms4,0,"Nah I don't think he goes to usf, he lives aro..."
sms5!,1,FreeMsg Hey there darling it's been 3 week's n...


In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize
tfidf_model = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs = tfidf_model.fit_transform(raw_documents=sms.text).toarray()

In [10]:
mask = sms.spam.astype(bool).values
spam_centroid = tfidf_docs[mask].mean(axis=0)
ham_centroid = tfidf_docs[~mask].mean(axis=0)

In [11]:
spamminess_score = tfidf_docs.dot(spam_centroid -ham_centroid)

In [12]:
from sklearn.preprocessing import MinMaxScaler
sms['lda_score'] = MinMaxScaler().fit_transform(spamminess_score.reshape(-1,1))
sms['lda_predict'] = (sms.lda_score > .5).astype(int)
sms['spam lda_predict lda_score'.split()].round(2).head(6)

,spam,lda_predict,lda_score
sms0,0,0,0.23
sms1,0,0,0.18
sms2!,1,1,0.72
sms3,0,0,0.18
sms4,0,0,0.29
sms5!,1,1,0.55


In [13]:
(1. - (sms.spam - sms.lda_predict).abs().sum() / len(sms)).round(3)

0.977

In [14]:
'''分類是不是垃圾郵件，先算出垃圾郵件與非垃圾郵件各自的質心
   將兩個質心相減算出距離，以0.5為標準(主要是判斷算出的距離中垃圾郵件與非垃圾郵件所佔的比例)
   (sms.spam - sms.lda_predict).abs計算出錯誤的預測數
   再除以長度算出錯誤率，1-錯誤率極為正確率'''

'分類是不是垃圾郵件，先算出垃圾郵件與非垃圾郵件各自的質心\n   將兩個質心相減算出距離，以0.5為標準(主要是判斷算出的距離中垃圾郵件與非垃圾郵件所佔的比例)\n   (sms.spam - sms.lda_predict).abs計算出錯誤的預測數\n   再除以長度算出錯誤率，1-錯誤率極為正確率'

In [15]:
from pugnlp.stats import Confusion
Confusion(sms['spam lda_predict'.split()])

C:\Anaconda\python\lib\site-packages\pugnlp\stats.py:504: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.__setattr__('_hist_labels', self.sum().astype(int))
C:\Anaconda\python\lib\site-packages\pugnlp\stats.py:510: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  setattr(self, '_hist_classes', self.T.sum())


lda_predict,0,1
spam,,
0,4135,64
1,45,593


In [16]:
with open('cats_and_dogs_sorted.txt', 'r', encoding="utf-8") as f:
    raw_data = []
    for line in f:
        raw_data.append(line.replace("\n", ""))

In [17]:
from nlpia.data.loaders import get_data
from nltk.tokenize import casual_tokenize
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
# from nltk.stem import PorterStemmer
from sklearn.decomposition import PCA
from nlpia.constants import DATA_PATH


NUM_TOPICS = 3
NUM_WORDS = 6
NUM_DOCS = NUM_PRETTY = 16
SAVE_SORTED_CORPUS = ''  # 'cats_and_dogs_sorted.txt'
# import nltk
# nltk.download('wordnet')  # noqa
# from nltk.stem.wordnet import WordNetLemmatizer


# STOPWORDS = 'a an and or the do are with from for of on in by if at to into them'.split()
# STOPWORDS += 'to at it its it\'s that than our you your - -- " \' ? , . !'.split()
STOPWORDS = []

# SYNONYMS = dict(zip(
#     'wolv people person women woman man human he  we  her she him his hers'.split(),
#     'wolf her    her    her   her   her her   her her her her her her her'.split()))
# SYNONYMS.update(dict(zip(
#     'ate pat smarter have had isn\'t hasn\'t no  got get become been was were wa be sat seat sit'.split(),
#     'eat pet smart   has  has not    not     not has has is     is   is  is   is is sit sit  sit'.split())))
# SYNONYMS.update(dict(zip(
#     'i me my mine our ours catbird bird birds birder tortoise turtle turtles turtle\'s don\'t'.split(),
#     'i i  i  i    i   i    bird    bird birds bird   turtle   turtle turtle  turtle    not'.split())))
SYNONYMS = {}

stemmer = None  # PorterStemmer()

pd.options.display.width = 110
pd.options.display.max_columns = 14
pd.options.display.max_colwidth = 32




def normalize_corpus_words(corpus, stemmer=stemmer, synonyms=SYNONYMS, stopwords=STOPWORDS):
    docs = [doc.lower() for doc in corpus]
    docs = [casual_tokenize(doc) for doc in docs]
    docs = [[synonyms.get(w, w) for w in words if w not in stopwords] for words in docs]
    if stemmer:
        docs = [[stemmer.stem(w) for w in words if w not in stopwords] for words in docs]
    docs = [[synonyms.get(w, w) for w in words if w not in stopwords] for words in docs]
    docs = [' '.join(w for w in words if w not in stopwords) for words in docs]
    return docs


def tokenize(text, vocabulary, synonyms=SYNONYMS, stopwords=STOPWORDS):
    doc = normalize_corpus_words([text.lower()], synonyms=synonyms, stopwords=stopwords)[0]
    stems = [w for w in doc.split() if w in vocabulary]
    return stems


fun_words = vocabulary = 'cat dog apple lion nyc love big small'
fun_stems = normalize_corpus_words([fun_words])[0].split()[:NUM_WORDS]
fun_words = fun_words.split()


if SAVE_SORTED_CORPUS:
    tfidfer = TfidfVectorizer(min_df=2, max_df=.6, stop_words=None, token_pattern=r'(?u)\b\w+\b')

    corpus = get_data('cats_and_dogs')[:NUM_DOCS]
    docs = normalize_corpus_words(corpus, stemmer=None)
    tfidf_dense = pd.DataFrame(tfidfer.fit_transform(docs).todense())
    id_words = [(i, w) for (w, i) in tfidfer.vocabulary_.items()]
    tfidf_dense.columns = list(zip(*sorted(id_words)))[1]


    word_tfidf_dense = pd.DataFrame(tfidfer.transform(fun_stems).todense())
    word_tfidf_dense.columns = list(zip(*sorted(id_words)))[1]
    word_tfidf_dense.index = fun_stems
    """
    >>> word_tfidf_dense[fun_stems]
          cat  dog  anim  pet  citi  appl  nyc  car  bike  hat
    cat   1.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0   0.0  0.0
    dog   0.0  1.0   0.0  0.0   0.0   0.0  0.0  0.0   0.0  0.0
    anim  0.0  0.0   1.0  0.0   0.0   0.0  0.0  0.0   0.0  0.0
    pet   0.0  0.0   0.0  1.0   0.0   0.0  0.0  0.0   0.0  0.0
    citi  0.0  0.0   0.0  0.0   1.0   0.0  0.0  0.0   0.0  0.0
    appl  0.0  0.0   0.0  0.0   0.0   1.0  0.0  0.0   0.0  0.0
    nyc   0.0  0.0   0.0  0.0   0.0   0.0  1.0  0.0   0.0  0.0
    car   0.0  0.0   0.0  0.0   0.0   0.0  0.0  1.0   0.0  0.0
    bike  0.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0   1.0  0.0
    hat   0.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0   0.0  1.0
    """

    tfidfer.use_idf = False
    tfidfer.norm = None
    bow_dense = pd.DataFrame(tfidfer.fit_transform(docs).todense())
    bow_dense.columns = list(zip(*sorted(id_words)))[1]
    bow_dense = bow_dense.astype(int)
    tfidfer.use_idf = True
    tfidfer.norm = 'l2'


    """
    >>> tfidf_dense.shape
    (200, 170)
    """


    bow_pretty = bow_dense.copy()
    bow_pretty = bow_pretty[fun_stems]
    bow_pretty['text'] = corpus

    tfidf_pretty = tfidf_dense.copy()
    tfidf_pretty = tfidf_pretty[fun_stems]
    tfidf_pretty['diversity'] = tfidf_pretty[fun_stems].T.sum().values
    tfidf_pretty['text'] = corpus
    # tfidf_pretty['diversity'] = [(row.diversity or 0) / ((float(row.iloc[i % (len(row) - 2)] or 1) ** 2))
    #                              for i, row in tfidf_pretty.iterrows()]
    tfidf_pretty = tfidf_pretty.sort_values('diversity', ascending=False).round(2)
    with open(os.path.join(DATA_PATH, SAVE_SORTED_CORPUS), 'w') as fout:
        fout.write('\n'.join(list(tfidf_pretty.text.values)))

    for col in fun_stems:
        bow_pretty.loc[bow_pretty[col] == 0, col] = ''
    # print(bow_pretty.head())


# do it all over again on a tiny portion of the corpus and vocabulary
import os
domin=os.path.abspath(r'C:\Users\ChuangYu\Desktop\NLP')
info = os.path.join(domin,'cats_and_dogs_sorted.txt') #將路徑與檔名結合起來就是每個檔案的完整路徑
info = open(info,'r',encoding="utf-8") #讀取檔案內容

corpus=info.read().split("\n")
corpus=corpus[:NUM_PRETTY]
docs = normalize_corpus_words(corpus)
tfidfer = TfidfVectorizer(min_df=1, max_df=.99, stop_words=None, token_pattern=r'(?u)\b\w+\b',
                          vocabulary=fun_stems)
tfidf_dense = pd.DataFrame(tfidfer.fit_transform(docs).todense())
id_words = [(i, w) for (w, i) in tfidfer.vocabulary_.items()]
tfidf_dense.columns = list(zip(*sorted(id_words)))[1]
tfidfer.use_idf = False
tfidfer.norm = None
bow_dense = pd.DataFrame(tfidfer.fit_transform(docs).todense())
bow_dense.columns = list(zip(*sorted(id_words)))[1]
bow_dense = bow_dense.astype(int)
tfidfer.use_idf = True
tfidfer.norm = 'l2'
bow_pretty = bow_dense.copy()
bow_pretty = bow_pretty[fun_stems]
bow_pretty['text'] = corpus
for col in fun_stems:
    bow_pretty.loc[bow_pretty[col] == 0, col] = ''
# print(bow_pretty)
word_tfidf_dense = pd.DataFrame(tfidfer.transform(fun_stems).todense())
word_tfidf_dense.columns = list(zip(*sorted(id_words)))[1]
word_tfidf_dense.index = fun_stems

tfidf_pretty = tfidf_dense.copy()
tfidf_pretty = tfidf_pretty[fun_stems]
tfidf_pretty = tfidf_pretty.round(2)
for col in fun_stems:
    tfidf_pretty.loc[tfidf_pretty[col] == 0, col] = ''
# tfidf_pretty[:text]tfidf_pretty.text.str[:16]
# print(tfidf_pretty)


"""
>>> pd.options.display.width = 150
>>> pd.options.display.max_columns = 16
>>> tfidf_pretty = tfidf_dense.copy()
>>> tfidf_pretty = tfidf_pretty[['bike', 'cat', 'car', 'chase', 'dog', 'hat', 'i']].head(10).round(1)
>>> tfidf_pretty[tfidf_pretty == 0] = ''
>>> tfidf_pretty['...'] = ''
>>> tfidf_pretty['text'] = corpus[:10]
>>> tfidf_pretty
   dog  cat bear  pet  hat bike chase bark meow ...                                         text
0       0.4            0.9                                                    The Cat in the Hat
1       0.5                                                                   The cat ate a rat.
2       0.6                       0.8                           The cat chased my laser pointer.
3  0.3                      0.6   0.4  0.6               A dog chased my bike and barked loudly.
4  0.4                                 0.6                           I ran from the barking dog.
5  0.4                            0.5  0.6                        A dog chased the car, barking.
6       0.5                                 0.9                                   The Cat's Meow
7       0.3       0.4                       0.5      The cat meowed so I pet it until it purred.
8       0.5                                 0.9                 A cat meowed on the hot tin roof
9  0.5  0.4                                                      Cats and dogs playing together.
"""

tfidf_zeros = tfidf_dense.T.sum()[tfidf_dense.T.sum() == 0]
# print(tfidf_zeros)
"""
>>> tfidf_zeros = tfidf_dense.T.sum()[tfidf_dense.T.sum() == 0]
>>> tfidf_zeros
199    0.0
"""

[corpus[i] for i in tfidf_zeros.index]
"""
>>> [corpus[i] for i in tfidf_zeros.index]
[]

# ['I flew a kite.', 'He froze.']
"""

pcaer = PCA(n_components=NUM_TOPICS)

doc_topic_vectors = pd.DataFrame(pcaer.fit_transform(tfidf_dense.values), columns=['top{}'.format(i) for i in range(NUM_TOPICS)])
doc_topic_vectors['text'] = corpus
pd.options.display.max_colwidth = 55
# doc_topic_vectors.round(1)
"""
>>> doc_topic_vectors.round(1)
    topic_A  topic_B  topic_C  topic_D                                                             text
0      -0.2      0.1      0.5     -0.2  Animals don't drive cars, but my pet dog likes to stick his ...
1       0.1      0.1     -0.0     -0.3              The Cat in the Hat is not about an animal or a hat.
2       0.1     -0.2      0.5     -0.3                   Cats don't like riding into the city in a car.
3      -0.2     -0.5      0.0      0.3                      Dogs love to chase cars, trucks, and bikes.
4      -0.2     -0.3      0.2      0.1        Wild cats chase bikes and runners but not cars or trucks.
5      -0.1     -0.0      0.6     -0.2              Animals, including pets, don't like riding in cars.
6       0.6      0.0     -0.1      0.0                                 NYC is a city that never sleeps.
7       0.7      0.1      0.0      0.2                                  Come to NYC. See the Big Apple!
8       0.8      0.1      0.0      0.2                                   NYC is known as the Big Apple.
9       0.7      0.1     -0.0      0.1  NYC is the only city where you can hardly find a typical Ame...
10     -0.2     -0.1     -0.5     -0.3                                         It rained cats and dogs.
11     -0.4      0.5     -0.0      0.4                                               I love my pet cat.
12      0.3      0.1     -0.1      0.4                                      I love New York City (NYC).
13     -0.2      0.3      0.2     -0.2                                      He pet the dog on the head.
14     -0.2     -0.5      0.2      0.0                                         Dogs like to chase cars.
15     -0.2     -0.1     -0.3     -0.1                          The cat steered clear of the dog house.
16     -0.1     -0.4      0.3      0.1                                         The car had a bike rack.
"""

word_topic_vectors = pd.DataFrame(pcaer.transform(word_tfidf_dense.values),
                                  columns=['top{}'.format(i) for i in range(NUM_TOPICS)])
word_topic_vectors.index = fun_stems


"""
>>> tfidf_similarity = []
... topic_similarity = []
... for i in range(10):
...     topic_similarity.append((doc_topic_vectors.iloc[i] * doc_topic_vectors.iloc[i+1]).sum())
...     tfidf_similarity.append((tfidf_dense.iloc[i] * tfidf_dense.iloc[i+1]).sum())
>>> tfidf_pretty['tfidf_similar'] = tfidf_similarity
>>> tfidf_pretty['topic_similar'] = topic_similarity
>>> tfidf_pretty
  bike  cat  car chase  dog  hat    i ...                                     text  tfidf_similar  topic_similar
0       0.4                  0.9                                The Cat in the Hat       0.217066       0.208128
1       0.5                                                     The cat ate a rat.       0.000000      -0.085262
2  0.5                            0.6                         I rode my bike home.       0.000000      -0.075669
3            0.5                                         The car is in the garage.       0.267063       0.045623
4            0.5   0.5  0.4                               Dogs like to chase cars.       0.620088       0.214047
5                       0.4                            The post man likes our dog.       0.316328       0.164364
6                       0.3                  He refused to sleep in the dog house.       0.000000      -0.057364
7       0.6        0.8                                     The cat chased a mouse.       0.000000      -0.004849
8                       0.4       0.4           I was in the dog house last night.       0.488741       0.025706
9       0.3             0.3                The cat steered clear of the dog house.       0.000000       0.011857
"""


def tfidf_search(text, corpus=tfidf_dense, corpus_text=corpus):
    """ search for the most relevant document """
    tokens = tokenize(text, vocabulary=corpus.columns)
    tfidf_vector_query = np.array(tfidfer.transform([' '.join(tokens)]).todense())[0]
    query_series = pd.Series(tfidf_vector_query, index=corpus.columns)

    return corpus_text[query_series.dot(corpus.T).values.argmax()]


def topic_search(text, corpus=doc_topic_vectors, pcaer=pcaer, corpus_text=corpus):
    """ search for the most relevant document """
    tokens = tokenize(text, vocabulary=corpus.columns)
    tfidf_vector_query = np.array(tfidfer.transform([' '.join(tokens)]).todense())[0]
    topic_vector_query = pcaer.transform([tfidf_vector_query])
    query_series = pd.Series(topic_vector_query, index=corpus.columns)
    return corpus_text[query_series.dot(corpus.T).values.argmax()]


"""
tfidf_search('Hello world, do you have a cat?')

topic_search('Hello world, do you have a cat?')
# 'Do you have a pet?'

search('The quick brown fox jumped over the lazy dog')
# 'The dog sat on the floor.'

search('A dog barked at my car incessantly.')
# 'A dog chased the car, barking.'
tokenize('A dog barked at my car incessantly.')
# ['dog', 'bark', 'at', 'car', 'incessantli']

search('A Rotweiller barked at my car incessantly.')
# 'The cat hated getting in the car.'
tokenize('A Rotweiller barked at my car incessantly.')
# ['rotweil', 'bark', 'at', 'car', 'incessantli']

list(df.columns)
# ['ate', 'can', 'car', 'cat', 'chase', 'cute', 'die', 'dog', 'ferret', 'flower', 'hair',
# 'hat', 'have', 'it', 'kitten', 'pet', 'ran',
#   'squirrel', 'struck', 'took', 'tree', 'trick', 'turtl', 'up', 'vet', 'water'],
"""
U, Sigma, VT = np.linalg.svd(tfidf_dense.T)  # <1> Transpose the doc-word tfidf matrix, because SVD works on column vectors
S = Sigma.copy()
S[4:] = 0
doc_labels = ['doc{}'.format(i) for i in range(len(tfidf_dense))]
U_df = pd.DataFrame(U, index=fun_stems, columns=fun_stems)
VT_df = pd.DataFrame(VT, index=doc_labels, columns=doc_labels)
ndim = 2
truncated_tfidf = U[:, :ndim].dot(np.diag(Sigma)[:ndim, :ndim]).dot(VT.T[:, :ndim].T)
"""
The left singular vectors tell you how to "rotate" the TF-IDF vectors into the topic space, equivalent to creating topics

>>> U_df
       cat   dog  appl   nyc   car  bike   hat
cat  -0.53  0.01 -0.50  0.31 -0.49 -0.00 -0.36
dog  -0.60  0.25  0.19  0.43  0.56 -0.00  0.21
appl -0.16 -0.63  0.17 -0.12  0.37 -0.00 -0.63
nyc  -0.25 -0.69  0.06  0.04 -0.24  0.00  0.63
car  -0.35  0.17  0.32 -0.45 -0.21  0.71 -0.03
bike -0.35  0.17  0.32 -0.45 -0.21 -0.71 -0.03
hat  -0.17  0.00 -0.69 -0.55  0.40  0.00  0.19

>>> VT_df.round(2)
        doc0  doc1  doc2  doc3  doc4  doc5  doc6  doc7  doc8  doc9  doc10  doc11
doc0  -0.37 -0.34 -0.16 -0.22 -0.33 -0.33 -0.27 -0.15 -0.40 -0.40  -0.15  -0.15
doc1   0.19  0.12  0.00  0.01  0.16  0.16 -0.29 -0.51  0.11  0.11  -0.51  -0.51
doc2   0.33  0.11 -0.55 -0.58  0.25  0.25 -0.19  0.11 -0.13 -0.13   0.11   0.11
doc3  -0.31 -0.39 -0.39 -0.27 -0.07 -0.07  0.21 -0.06  0.48  0.48  -0.06  -0.06
doc4   0.03 -0.59  0.28  0.08  0.24  0.24 -0.61  0.14  0.10  0.10   0.14   0.14
doc5   0.00  0.00  0.00 -0.00  0.71 -0.71 -0.00 -0.00  0.00  0.00  -0.00  -0.00
doc6   0.16 -0.51  0.17 -0.10  0.27  0.27  0.62 -0.11 -0.23 -0.23  -0.11  -0.11
doc7  -0.04 -0.07 -0.36  0.41  0.09  0.09  0.00  0.67 -0.06 -0.06  -0.33  -0.33
doc8  -0.54  0.19  0.07 -0.08  0.27  0.27  0.00 -0.00  0.45 -0.55  -0.00  -0.00
doc9  -0.54  0.19  0.07 -0.08  0.27  0.27  0.00 -0.00 -0.55  0.45  -0.00  -0.00
doc10 -0.04 -0.07 -0.36  0.41  0.09  0.09  0.00 -0.33 -0.06 -0.06   0.67  -0.33
doc11 -0.04 -0.07 -0.36  0.41  0.09  0.09  0.00 -0.33 -0.06 -0.06  -0.33   0.67
Try to reconstruct an approximate TFIDF, using only 2 topics (from 7 words):

>>> tfidf_compressed = U[:,:2] @ (pd.np.diag(S)[:2,:] @ VT[:2,:])
>>> tfidf_compressed.shape

array([[ 0.12191697,  0.01013273,  0.04009995, ...,  0.06057937,
         0.07675374, -0.00521042],
       [ 0.0352883 ,  0.07193914,  0.00248612, ...,  0.00928599,
         0.02413519,  0.02511732],
       [ 0.0886943 ,  0.0689894 ,  0.06191171, ...,  0.08041404,
        -0.0090325 ,  0.01368156],
       ...,
       [-0.00116523,  0.00980797,  0.00397578, ..., -0.00540478,
         0.04008382,  0.01717131],
       [ 0.08695312,  0.01880789,  0.04888827, ...,  0.0604496 ,
         0.04363558,  0.00425347],
       [ 0.06353676, -0.01267888,  0.0766847 , ...,  0.07791765,
         0.01870914,  0.00097211]])
"""


if __name__ == '__main__':
    print(word_topic_vectors.T.round(1))

      cat  dog  apple  lion  nyc  love
top0 -0.6 -0.4    0.5  -0.3  0.4  -0.1
top1 -0.1 -0.3   -0.4  -0.1  0.1   0.8
top2 -0.3  0.8   -0.1  -0.5  0.0   0.1


In [18]:
word_topic_vectors.T.round(1)

,cat,dog,apple,lion,nyc,love
top0,-0.6,-0.4,0.5,-0.3,0.4,-0.1
top1,-0.1,-0.3,-0.4,-0.1,0.1,0.8
top2,-0.3,0.8,-0.1,-0.5,0.0,0.1


### 4.3 Singular value decomposition

In [19]:
""" Uses np.linalg.svd directly to illustrate LSA on a small corpus. 

Examples from the SVD section in Chapter 4 of NLPIA

SEE ALSO: ch04_stanford_lsa.py

>>> from nlpia.book.examples.ch04_catdog_lsa_sorted import lsa_models, prettify_tdm
>>> bow_svd, tfidf_svd = lsa_models()  # <1>
>>> prettify_tdm(**bow_svd)
   cat dog apple lion nyc love                                             text
0              1        1                                 NYC is the Big Apple.
1              1        1                        NYC is known as the Big Apple.
2                       1    1                                      I love NYC!
3              1        1           I wore a hat to the Big Apple party in NYC.
4              1        1                       Come to NYC. See the Big Apple!
5              1                             Manhattan is called the Big Apple.
6    1                                  New York is a big city for a small cat.
7    1              1           The lion, a big cat, is the king of the jungle.
8    1                       1                               I love my pet cat.
9                       1    1                      I love New York City (NYC).
10   1   1                                              Your dog chased my cat.
>>> tdm = bow_svd['tdm']
>>> tdm

       0   1   2   3   4   5   6   7   8   9   10
cat     0   0   0   0   0   0   1   1   1   0   1
dog     0   0   0   0   0   0   0   0   0   0   1
apple   1   1   0   1   1   1   0   0   0   0   0
lion    0   0   0   0   0   0   0   1   0   0   0
nyc     1   1   1   1   1   0   0   0   0   1   0
love    0   0   1   0   0   0   0   0   1   1   0
>>> import numpy as np
>>> U, s, Vt = np.linalg.svd(tdm)  # <1>
 
>>> import pandas as pd
>>> pd.DataFrame(U, index=tdm.index).round(2)
          0     1     2     3     4     5
cat   -0.04  0.83 -0.38 -0.00  0.11 -0.38
dog   -0.00  0.21 -0.18 -0.71 -0.39  0.52
apple -0.62 -0.21 -0.51  0.00  0.49  0.27
lion  -0.00  0.21 -0.18  0.71 -0.39  0.52
nyc   -0.75 -0.00  0.24 -0.00 -0.52 -0.32
love  -0.22  0.42  0.69  0.00  0.41  0.37

>>> s.round(1)
array([3.1, 2.2, 1.8, 1. , 0.8, 0.5])
>>> S = np.zeros((len(U), len(Vt)))
>>> pd.np.fill_diagonal(S, s)
>>> pd.DataFrame(S).round(1)
    0    1    2    3    4    5    6    7    8    9    10
0  3.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1  0.0  2.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  0.0  0.0  1.8  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4  0.0  0.0  0.0  0.0  0.8  0.0  0.0  0.0  0.0  0.0  0.0
5  0.0  0.0  0.0  0.0  0.0  0.5  0.0  0.0  0.0  0.0  0.0

>>> pd.DataFrame(Vt).round(2)
      0     1     2     3     4     5     6     7     8     9     10
0  -0.44 -0.44 -0.31 -0.44 -0.44 -0.20 -0.01 -0.01 -0.08 -0.31 -0.01
1  -0.09 -0.09  0.19 -0.09 -0.09 -0.09  0.37  0.47  0.56  0.19  0.47
2  -0.16 -0.16  0.52 -0.16 -0.16 -0.29 -0.22 -0.32  0.17  0.52 -0.32
3   0.00 -0.00 -0.00  0.00  0.00  0.00 -0.00  0.71  0.00 -0.00 -0.71
4  -0.04 -0.04 -0.14 -0.04 -0.04  0.58  0.13 -0.33  0.62 -0.14 -0.33
5  -0.09 -0.09  0.10 -0.09 -0.09  0.51 -0.73  0.27 -0.01  0.10  0.27
6  -0.57  0.21  0.11  0.33 -0.31  0.34  0.34 -0.00 -0.34  0.23  0.00
7  -0.32  0.47  0.25 -0.63  0.41  0.07  0.07  0.00 -0.07 -0.18  0.00
8  -0.50  0.29 -0.20  0.41  0.16 -0.37 -0.37 -0.00  0.37 -0.17  0.00
9  -0.15 -0.15 -0.59 -0.15  0.42  0.04  0.04 -0.00 -0.04  0.63 -0.00
10 -0.26 -0.62  0.33  0.24  0.54  0.09  0.09 -0.00 -0.09 -0.23 -0.00

>>> tdm = bow_svd['tdm']
>>> U, s, Vt = np.linalg.svd(tdm)
>>> S = np.zeros((len(U), len(Vt)))
>>> np.fill_diagonal(S, s)
>>> err = [0]
>>> for numdim in range(len(s), 0, -1):
...     S[numdim - 1, numdim - 1] = 0
...     reconstructed_tdm = U.dot(S).dot(Vt)
...     err.append(np.sqrt(((reconstructed_tdm - tdm).values.flatten() ** 2).sum() 
...                / np.product(tdm.shape)))
>>> np.array(err).round(2)
array([0.  , 0.06, 0.12, 0.17, 0.28, 0.39, 0.55])

>>> tdm = tfidf_svd['tdm']
>>> U, s, Vt = np.linalg.svd(tdm)
>>> S = np.zeros((len(U), len(Vt)))
>>> np.fill_diagonal(S, s)
>>> err2 = [0]
>>> for numdim in range(len(s), 0, -1):
...     S[numdim - 1, numdim - 1] = 0
...     reconstructed_tdm = U.dot(S).dot(Vt)
...     err2.append(np.sqrt(((reconstructed_tdm - tdm).values.flatten() ** 2).sum() 
...                / np.product(tdm.shape)))
>>> np.array(err2).round(2)
array([0.  , 0.07, 0.11, 0.15, 0.23, 0.3 , 0.41])
"""
import sys
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('TkAgg')  # noqa
import seaborn  # noqa
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

from nlpia.data.loaders import get_data

pd.options.display.width = 120
pd.options.display.max_columns = 16

VOCABULARY = vocabulary='cat dog apple lion NYC love'.lower().split()  # 'cat dog apple lion NYC love big small bright'.lower().split()
with open('cats_and_dogs_sorted.txt', 'r', encoding="utf-8") as f:
    raw_data = []
    for line in f:
        raw_data.append(line.replace("\n", ""))
DOCS = raw_data


def docs_to_tdm(docs=DOCS, vocabulary=VOCABULARY, verbosity=0):
    tfidfer = TfidfVectorizer(min_df=1, max_df=.99, stop_words=None, token_pattern=r'(?u)\b\w+\b',
                              vocabulary=vocabulary)
    tfidf_dense = pd.DataFrame(tfidfer.fit_transform(docs).todense())
    id_words = [(i, w) for (w, i) in tfidfer.vocabulary_.items()]
    tfidf_dense.columns = list(zip(*sorted(id_words)))[1]

    tfidfer.use_idf = False
    tfidfer.norm = None
    bow_dense = pd.DataFrame(tfidfer.fit_transform(docs).todense())
    bow_dense.columns = list(zip(*sorted(id_words)))[1]
    bow_dense = bow_dense.astype(int)
    tfidfer.use_idf = True
    tfidfer.norm = 'l2'
    if verbosity:
        print(tfidf_dense.T)
    return bow_dense.T, tfidf_dense.T, tfidfer


def prettify_tdm(tdm=None, docs=[], vocabulary=[], **kwargs):
    bow_pretty = tdm.T.copy()[vocabulary]
    bow_pretty['text'] = docs
    for col in vocabulary:
        bow_pretty.loc[bow_pretty[col] == 0, col] = ''
    return bow_pretty


def accuracy_study(tdm=None, u=None, s=None, vt=None, verbosity=0, **kwargs):
    """ Reconstruct the term-document matrix and measure error as SVD terms are truncated
    """
    smat = np.zeros((len(u), len(vt)))
    np.fill_diagonal(smat, s)
    smat = pd.DataFrame(smat, columns=vt.index, index=u.index)
    if verbosity:
        print()
        print('Sigma:')
        print(smat.round(2))
        print()
        print('Sigma without zeroing any dim:')
        print(np.diag(smat.round(2)))
    tdm_prime = u.values.dot(smat.values).dot(vt.values)
    if verbosity:
        print()
        print('Reconstructed Term-Document Matrix')
        print(tdm_prime.round(2))

    err = [np.sqrt(((tdm_prime - tdm).values.flatten() ** 2).sum() / np.product(tdm.shape))]
    if verbosity:
        print()
        print('Error without reducing dimensions:')
        print(err[-1])
    # 2.3481474529927113e-15

    smat2 = smat.copy()
    for numdim in range(len(s) - 1, 0, -1):
        smat2.iloc[numdim, numdim] = 0
        if verbosity:
            print('Sigma after zeroing out dim {}'.format(numdim))
            print(np.diag(smat2.round(2)))
            #           d0    d1   d2   d3   d4   d5
            # ship    2.16  0.00  0.0  0.0  0.0  0.0
            # boat    0.00  1.59  0.0  0.0  0.0  0.0
            # ocean   0.00  0.00  0.0  0.0  0.0  0.0
            # voyage  0.00  0.00  0.0  0.0  0.0  0.0
            # trip    0.00  0.00  0.0  0.0  0.0  0.0

        tdm_prime2 = u.values.dot(smat2.values).dot(vt.values)
        err += [np.sqrt(((tdm_prime2 - tdm).values.flatten() ** 2).sum() / np.product(tdm.shape))]
        if verbosity:
            print('Error after zeroing out dim {}'.format(numdim))
            print(err[-1])
    return err


def lsa(tdm, verbosity=0):
    if verbosity:
        print(tdm)
        #         0   1   2   3   4   5   6   7   8   9   10
        # cat     0   0   0   0   0   0   1   1   1   0   1
        # dog     0   0   0   0   0   0   0   0   0   0   1
        # apple   1   1   0   1   1   1   0   0   0   0   0
        # lion    0   0   0   0   0   0   0   1   0   0   0
        # love    0   0   1   0   0   0   0   0   1   1   0
        # nyc     1   1   1   1   1   0   0   0   0   1   0

    u, s, vt = np.linalg.svd(tdm)

    u = pd.DataFrame(u, index=tdm.index)
    if verbosity:
        print('U')
        print(u.round(2))
        # U
        #           0     1     2     3     4     5
        # cat   -0.04  0.83 -0.38 -0.00  0.11  0.38
        # dog   -0.00  0.21 -0.18 -0.71 -0.39 -0.52
        # apple -0.62 -0.21 -0.51  0.00  0.49 -0.27
        # lion  -0.00  0.21 -0.18  0.71 -0.39 -0.52
        # love  -0.22  0.42  0.69  0.00  0.41 -0.37
        # nyc   -0.75  0.00  0.24 -0.00 -0.52  0.32

    vt = pd.DataFrame(vt, index=['d{}'.format(i) for i in range(len(vt))])
    if verbosity:
        print('VT')
        print(vt.round(2))
        # VT
        #        0     1     2     3     4     5     6     7     8     9     10
        # d0  -0.44 -0.44 -0.31 -0.44 -0.44 -0.20 -0.01 -0.01 -0.08 -0.31 -0.01
        # d1  -0.09 -0.09  0.19 -0.09 -0.09 -0.09  0.37  0.47  0.56  0.19  0.47
        # d2  -0.16 -0.16  0.52 -0.16 -0.16 -0.29 -0.22 -0.32  0.17  0.52 -0.32
        # d3   0.00 -0.00  0.00  0.00  0.00  0.00 -0.00  0.71  0.00  0.00 -0.71
        # d4  -0.04 -0.04 -0.14 -0.04 -0.04  0.58  0.13 -0.33  0.62 -0.14 -0.33
        # d5   0.09  0.09 -0.10  0.09  0.09 -0.51  0.73 -0.27  0.01 -0.10 -0.27
        # d6  -0.55  0.24  0.15  0.36 -0.38  0.32  0.32  0.00 -0.32  0.17  0.00
        # d7  -0.32  0.46  0.23 -0.64  0.41  0.09  0.09  0.00 -0.09 -0.14  0.00
        # d8  -0.52  0.27 -0.24  0.39  0.22 -0.36 -0.36 -0.00  0.36 -0.12  0.00
        # d9  -0.14 -0.14 -0.58 -0.14  0.32  0.10  0.10 -0.00 -0.10  0.68 -0.00
        # d10 -0.27 -0.63  0.31  0.23  0.55  0.12  0.12 -0.00 -0.12 -0.19 -0.00

    # Reconstruct the original term-document matrix.
    # The sum of the squares of the error is 0.

    return {'u': u, 's': s, 'vt': vt, 'tdm': tdm}


def plot_feature_selection(accuracy, title=None):
    # accuracy = topic_model['accuracy']
    plt.plot(range(len(accuracy)), accuracy)
    plt.title(title or 'LSA Model Accuracy')
    plt.xlabel('Number of Dimensions Eliminated')
    plt.ylabel('Reconstruction Accuracy')
    plt.grid(True)
    plt.tight_layout()
    return accuracy


""" Some more complicated examples of SVD.
>>> import numpy as np
>>> u, s, vt = np.linalg.svd(tdm)  # <1>

>>> import pandas as pd
>>> u = pd.DataFrame(u, index=tdm.index)
>>> u.round(2)


>>> main('cat dog apple lion NYC love'.lower().split())
263it [00:00, 408405.02it/s]
             0         1         2         3         4    5    6         7         8         9         10   11
nyc    0.674278  0.674278  0.596469  0.674278  0.674278  0.0  0.0  0.000000  0.000000  0.596469  0.000000  0.0
lion   0.000000  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.826567  0.000000  0.000000  0.000000  0.0
love   0.000000  0.000000  0.802636  0.000000  0.000000  0.0  0.0  0.000000  0.744190  0.802636  0.000000  0.0
dog    0.000000  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.000000  0.000000  0.000000  0.826567  0.0
apple  0.738477  0.738477  0.000000  0.738477  0.738477  1.0  0.0  0.000000  0.000000  0.000000  0.000000  0.0
cat    0.000000  0.000000  0.000000  0.000000  0.000000  0.0  1.0  0.562839  0.667968  0.000000  0.562839  0.0
   nyc lion love dog apple cat                                             text
0    1                   1                                NYC is the Big Apple.
1    1                   1                       NYC is known as the Big Apple.
2    1         1                                                    I love NYC!
3    1                   1          I wore a hat to the Big Apple party in NYC.
4    1                   1                      Come to NYC. See the Big Apple!
5                        1                   Manhattan is called the Big Apple.
6                            1          New York is a big city for a small cat.
7         1                  1  The lion, a big cat, is the king of the jungle.
8              1             1                               I love my pet cat.
9    1         1                                    I love New York City (NYC).
10                 1         1                          Your dog chased my cat.
11                                                     Bright lights, big city?
       0   1   2   3   4   5   6   7   8   9   10  11
nyc     1   1   1   1   1   0   0   0   0   1   0   0
lion    0   0   0   0   0   0   0   1   0   0   0   0
love    0   0   1   0   0   0   0   0   1   1   0   0
dog     0   0   0   0   0   0   0   0   0   0   1   0
apple   1   1   0   1   1   1   0   0   0   0   0   0
cat     0   0   0   0   0   0   1   1   1   0   1   0
U
          0     1     2     3     4     5
nyc   -0.75 -0.00  0.24  0.00 -0.52 -0.32
lion  -0.00  0.21 -0.18 -0.71 -0.39  0.52
love  -0.22  0.42  0.69 -0.00  0.41  0.37
dog   -0.00  0.21 -0.18  0.71 -0.39  0.52
apple -0.62 -0.21 -0.51 -0.00  0.49  0.27
cat   -0.04  0.83 -0.38 -0.00  0.11 -0.38
VT
       0     1     2     3     4     5     6     7     8     9     10   11
d0  -0.44 -0.44 -0.31 -0.44 -0.44 -0.20 -0.01 -0.01 -0.08 -0.31 -0.01  0.0
d1  -0.09 -0.09  0.19 -0.09 -0.09 -0.09  0.37  0.47  0.56  0.19  0.47  0.0
d2  -0.16 -0.16  0.52 -0.16 -0.16 -0.29 -0.22 -0.32  0.17  0.52 -0.32  0.0
d3   0.00  0.00  0.00  0.00  0.00 -0.00 -0.00 -0.71 -0.00  0.00  0.71  0.0
d4  -0.04 -0.04 -0.14 -0.04 -0.04  0.58  0.13 -0.33  0.62 -0.14 -0.33  0.0
d5  -0.09 -0.09  0.10 -0.09 -0.09  0.51 -0.73  0.27 -0.01  0.10  0.27  0.0
d6   0.12 -0.03  0.20 -0.03 -0.51  0.45  0.45  0.00 -0.45  0.25  0.00  0.0
d7   0.45 -0.85 -0.04  0.15  0.22  0.02  0.02  0.00 -0.02  0.06  0.00  0.0
d8   0.52  0.15 -0.38  0.15 -0.59 -0.24 -0.24  0.00  0.24  0.14  0.00  0.0
d9  -0.28 -0.01 -0.62 -0.01  0.22  0.07  0.07  0.00 -0.07  0.69  0.00  0.0
d10  0.45  0.15 -0.04 -0.85  0.22  0.02  0.02  0.00 -0.02  0.06  0.00  0.0
d11  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  1.0
Sigma
         d0    d1    d2   d3    d4    d5   d6   d7   d8   d9  d10  d11
nyc    3.14  0.00  0.00  0.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0
lion   0.00  2.24  0.00  0.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0
love   0.00  0.00  1.77  0.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0
dog    0.00  0.00  0.00  1.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0
apple  0.00  0.00  0.00  0.0  0.84  0.00  0.0  0.0  0.0  0.0  0.0  0.0
cat    0.00  0.00  0.00  0.0  0.00  0.52  0.0  0.0  0.0  0.0  0.0  0.0
Sigma without zeroing any dim
[3.14 2.24 1.77 1.   0.84 0.52]
Reconstructed Term-Document Matrix
[[ 1.  1.  1.  1.  1.  0. -0. -0. -0.  1. -0.  0.]
 [-0. -0.  0. -0. -0. -0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0. -0. -0.  1.  1. -0.  0.]
 [-0. -0.  0. -0. -0. -0.  0.  0.  0.  0.  1.  0.]
 [ 1.  1.  0.  1.  1.  1.  0.  0.  0.  0.  0.  0.]
 [-0. -0.  0. -0. -0. -0.  1.  1.  1.  0.  1.  0.]]
Error without reducing dimensions
1.5846963069762592e-15
Sigma after zeroing out dim 5
[3.14 2.24 1.77 1.   0.84 0.  ]
Error after zeroing out dim 5
0.0614920369471735
Sigma after zeroing out dim 4
[3.14 2.24 1.77 1.   0.   0.  ]
Error after zeroing out dim 4
0.11683863316404541
Sigma after zeroing out dim 3
[3.14 2.24 1.77 0.   0.   0.  ]
Error after zeroing out dim 3
0.1659522675004209
Sigma after zeroing out dim 2
[3.14 2.24 0.   0.   0.   0.  ]
Error after zeroing out dim 2
0.2667342349285279
Sigma after zeroing out dim 1
[3.14 0.   0.   0.   0.   0.  ]
Error after zeroing out dim 1
0.3749554593913143
[1.         0.93850796 0.88316137 0.83404773 0.73326577 0.62504454]
             0         1         2         3         4    5    6         7         8         9         10   11
nyc    0.674278  0.674278  0.596469  0.674278  0.674278  0.0  0.0  0.000000  0.000000  0.596469  0.000000  0.0
lion   0.000000  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.826567  0.000000  0.000000  0.000000  0.0
love   0.000000  0.000000  0.802636  0.000000  0.000000  0.0  0.0  0.000000  0.744190  0.802636  0.000000  0.0
dog    0.000000  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.000000  0.000000  0.000000  0.826567  0.0
apple  0.738477  0.738477  0.000000  0.738477  0.738477  1.0  0.0  0.000000  0.000000  0.000000  0.000000  0.0
cat    0.000000  0.000000  0.000000  0.000000  0.000000  0.0  1.0  0.562839  0.667968  0.000000  0.562839  0.0
U
          0     1     2     3     4     5
nyc   -0.66  0.07 -0.27 -0.00  0.51 -0.47
lion  -0.00  0.18  0.21  0.71  0.48  0.44
love  -0.21  0.54 -0.64  0.00 -0.30  0.41
dog   -0.00  0.18  0.21 -0.71  0.48  0.44
apple -0.72 -0.26  0.40 -0.00 -0.38  0.33
cat   -0.04  0.76  0.52 -0.00 -0.19 -0.35
VT
       0     1     2     3     4     5     6     7     8     9     10   11
d0  -0.44 -0.44 -0.25 -0.44 -0.44 -0.32 -0.02 -0.01 -0.08 -0.25 -0.01  0.0
d1  -0.09 -0.09  0.29 -0.09 -0.09 -0.16  0.46  0.35  0.56  0.29  0.35  0.0
d2   0.08  0.08 -0.50  0.08  0.08  0.30  0.38  0.34 -0.09 -0.50  0.34  0.0
d3  -0.00 -0.00  0.00 -0.00 -0.00  0.00 -0.00  0.71  0.00  0.00 -0.71  0.0
d4   0.09  0.09  0.10  0.09  0.09 -0.54 -0.27  0.41 -0.49  0.10  0.41  0.0
d5  -0.13 -0.13  0.08 -0.13 -0.13  0.58 -0.62  0.30  0.13  0.08  0.30  0.0
d6   0.22  0.01  0.20  0.01 -0.64  0.30  0.33  0.00 -0.49  0.26  0.00  0.0
d7   0.46 -0.85 -0.03  0.15  0.21  0.02  0.02  0.00 -0.03  0.06  0.00  0.0
d8   0.49  0.17 -0.42  0.17 -0.49 -0.24 -0.27  0.00  0.40  0.05  0.00  0.0
d9  -0.24 -0.01 -0.61 -0.01  0.17  0.07  0.07  0.00 -0.11  0.72  0.00  0.0
d10  0.46  0.15 -0.03 -0.85  0.21  0.02  0.02  0.00 -0.03  0.06  0.00  0.0
d11  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  1.0
Sigma
         d0    d1    d2    d3    d4    d5   d6   d7   d8   d9  d10  d11
nyc    2.24  0.00  0.00  0.00  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0
lion   0.00  1.63  0.00  0.00  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0
love   0.00  0.00  1.36  0.00  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0
dog    0.00  0.00  0.00  0.83  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0
apple  0.00  0.00  0.00  0.00  0.71  0.00  0.0  0.0  0.0  0.0  0.0  0.0
cat    0.00  0.00  0.00  0.00  0.00  0.56  0.0  0.0  0.0  0.0  0.0  0.0
Sigma without zeroing any dim
[2.24 1.63 1.36 0.83 0.71 0.56]
Reconstructed Term-Document Matrix
[[ 0.67  0.67  0.6   0.67  0.67  0.   -0.    0.    0.    0.6   0.    0.  ]
 [ 0.    0.   -0.    0.    0.    0.    0.    0.83  0.   -0.   -0.    0.  ]
 [ 0.    0.    0.8   0.    0.   -0.    0.    0.    0.74  0.8   0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.83  0.  ]
 [ 0.74  0.74  0.    0.74  0.74  1.   -0.   -0.    0.   -0.   -0.    0.  ]
 [-0.   -0.    0.   -0.   -0.    0.    1.    0.56  0.67  0.    0.56  0.  ]]
Error without reducing dimensions
2.4118514282911836e-16
Sigma after zeroing out dim 5
[2.24 1.63 1.36 0.83 0.71 0.  ]
Error after zeroing out dim 5
0.06622832691855576
Sigma after zeroing out dim 4
[2.24 1.63 1.36 0.83 0.   0.  ]
Error after zeroing out dim 4
0.10640521215974559
Sigma after zeroing out dim 3
[2.24 1.63 1.36 0.   0.   0.  ]
Error after zeroing out dim 3
0.14426065117971226
Sigma after zeroing out dim 2
[2.24 1.63 0.   0.   0.   0.  ]
Error after zeroing out dim 2
0.21538404738257216
Sigma after zeroing out dim 1
[2.24 0.   0.   0.   0.   0.  ]
Error after zeroing out dim 1
0.28856858954287745
[1.         0.93377167 0.89359479 0.85573935 0.78461595 0.71143141]
"""


def lsa_models(vocabulary='cat dog apple lion NYC love'.lower().split(), docs=11, verbosity=0):
    # vocabulary = 'cat dog apple lion NYC love big small bright'.lower().split()
    if isinstance(docs, int):
        docs = raw_datas[:docs]
    tdm, tfidfdm, tfidfer = docs_to_tdm(docs=docs, vocabulary=vocabulary)
    lsa_bow_model = lsa(tdm)  # (tdm - tdm.mean(axis=1)) # SVD fails to converge if you center, like PCA does
    lsa_bow_model['vocabulary'] = tdm.index.values
    lsa_bow_model['docs'] = docs
    err = accuracy_study(verbosity=verbosity, **lsa_bow_model)
    lsa_bow_model['err'] = err
    lsa_bow_model['accuracy'] = list(1. - np.array(err))
    
    lsa_tfidf_model = lsa(tdm=tfidfdm)
    lsa_bow_model['vocabulary'] = tfidfdm.index.values
    lsa_tfidf_model['docs'] = docs
    err = accuracy_study(verbosity=verbosity, **lsa_tfidf_model)
    lsa_tfidf_model['err'] = err
    lsa_tfidf_model['accuracy'] = list(1. - np.array(err))

    return lsa_bow_model, lsa_tfidf_model


if __name__ == '__main__':
    numdocs = 11
    with open('cats_and_dogs_sorted.txt', 'r', encoding="utf-8") as f:
        raw_datas =[]
        for line in f:
            raw_datas.append(line.replace("\n", ""))
    docs = raw_datas[:numdocs]
    vocabulary = sys.argv[1:] or 'cat dog apple lion NYC love'.lower().split()
    lsa_bow_model, lsa_tfidf_model = lsa_models(vocabulary=vocabulary, docs=docs)
    tdm = lsa_bow_model['tdm']
    tfidfdm = lsa_tfidf_model['tdm']
    print(prettify_tdm(tdm=tdm, docs=docs, vocabulary=vocabulary))
    acc = plot_feature_selection(accuracy=lsa_bow_model['accuracy'])
    print("BOW accuracy after multiplying Truncated SVD back together")
    print(acc)
    acc = plot_feature_selection(accuracy=lsa_tfidf_model['accuracy'], title='TF-IDF LSA Model Accuracy')
    print("TF-IDF accuracy after multiplying Truncated SVD back together")
    print(acc)
    plt.legend(['BOW Reconstruction Accuracy', 'TF-IDF Reconstruction Accuracy'])
    yn = input('Would you like to see the accuracy comparison plot? [y]')
    if not yn or yn.lower().startswith('y'):
        plt.show()


   -f C:\Users\ChuangYu\AppData\Roaming\jupyter\runtime\kernel-e82623fd-15f2-4d2b-897c-761ef2213ef1.json  \
0                                                                                                          
1                                                                                                          
2                                                                                                          
3                                                                                                          
4                                                                                                          
5                                                                                                          
6                                                                                                          
7                                                                                                          
8                           

In [20]:
bow_svd, tfidf_svd = lsa_models()
prettify_tdm(**bow_svd)

,cat,dog,apple,lion,nyc,love,text
0,,,1,,1,,NYC is the Big Apple.
1,,,1,,1,,NYC is known as the Big Apple.
2,,,,,1,1,I love NYC!
3,,,1,,1,,I wore a hat to the Big Apple party in NYC.
4,,,1,,1,,Come to NYC. See the Big Apple!
5,,,1,,,,Manhattan is called the Big Apple.
6,1,,,,,,New York is a big city for a small cat.
7,1,,,1,,,"The lion, a big cat, is the king of the jungle."
8,1,,,,,1,I love my pet cat.
9,,,,,1,1,I love New York City (NYC).


In [21]:
tdm = bow_svd['tdm']
tdm

,0,1,2,3,4,5,6,7,8,9,10
cat,0,0,0,0,0,0,1,1,1,0,1
dog,0,0,0,0,0,0,0,0,0,0,1
apple,1,1,0,1,1,1,0,0,0,0,0
lion,0,0,0,0,0,0,0,1,0,0,0
nyc,1,1,1,1,1,0,0,0,0,1,0
love,0,0,1,0,0,0,0,0,1,1,0


In [22]:
import numpy as np
U, s, Vt = np.linalg.svd(tdm)
import pandas as pd
pd.DataFrame(U, index=tdm.index).round(2)

,0,1,2,3,4,5
cat,-0.04,0.83,-0.38,-0.00,0.11,-0.38
dog,-0.00,0.21,-0.18,-0.71,-0.39,0.52
apple,-0.62,-0.21,-0.51,0.00,0.49,0.27
lion,-0.00,0.21,-0.18,0.71,-0.39,0.52
nyc,-0.75,0.00,0.24,-0.00,-0.52,-0.32
love,-0.22,0.42,0.69,0.00,0.41,0.37


In [23]:
s.round(1)

array([3.1, 2.2, 1.8, 1. , 0.8, 0.5])

In [24]:
S = np.zeros((len(U), len(Vt)))
pd.np.fill_diagonal(S, s)
pd.DataFrame(S).round(1)

,0,1,2,3,4,5,6,7,8,9,10
0,3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,2.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0


In [25]:
pd.DataFrame(Vt).round(2)

,0,1,2,3,4,5,6,7,8,9,10
0,-0.44,-0.44,-0.31,-0.44,-0.44,-0.20,-0.01,-0.01,-0.08,-0.31,-0.01
1,-0.09,-0.09,0.19,-0.09,-0.09,-0.09,0.37,0.47,0.56,0.19,0.47
2,-0.16,-0.16,0.52,-0.16,-0.16,-0.29,-0.22,-0.32,0.17,0.52,-0.32
3,0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.71,-0.00,-0.00,-0.71
4,-0.04,-0.04,-0.14,-0.04,-0.04,0.58,0.13,-0.33,0.62,-0.14,-0.33
5,-0.09,-0.09,0.10,-0.09,-0.09,0.51,-0.73,0.27,-0.01,0.10,0.27
6,-0.57,0.21,0.11,0.33,-0.31,0.34,0.34,0.00,-0.34,0.23,0.00
7,-0.32,0.47,0.25,-0.63,0.41,0.07,0.07,0.00,-0.07,-0.18,0.00
8,-0.50,0.29,-0.20,0.41,0.16,-0.37,-0.37,-0.00,0.37,-0.17,0.00
9,-0.15,-0.15,-0.59,-0.15,0.42,0.04,0.04,-0.00,-0.04,0.63,-0.00


In [26]:
err = []
for numdim in range(len(s), 0, -1):
    S[numdim - 1, numdim - 1] = 0
    reconstructed_tdm = U.dot(S).dot(Vt)
    err.append(np.sqrt(((\
    reconstructed_tdm - tdm).values.flatten() ** 2).sum()/ np.product(tdm.shape)))
np.array(err).round(2)

array([0.06, 0.12, 0.17, 0.28, 0.39, 0.55])

In [27]:
import pandas as pd
pd.set_option('display.max_columns', 6)
from sklearn.decomposition import PCA
import seaborn
from matplotlib import pyplot as plt
from nlpia.data.loaders import get_data
df = get_data('pointcloud').sample(1000)
pca = PCA(n_components=2)
df2d = pd.DataFrame(pca.fit_transform(df), columns=list('xy'))
df2d.plot(kind='scatter', x='x', y='y')
plt.show()

INFO:nlpia.futil:Reading CSV with `read_csv(*('C:\\Anaconda\\python\\lib\\site-packages\\nlpia\\data\\pointcloud.csv.gz',), **{'nrows': None, 'low_memory': False})`...


In [28]:
import pandas as pd
from nlpia.data.loaders import get_data
pd.options.display.width = 120
sms = get_data('sms-spam')
index = ['sms{}{}'.format(i, '!'*j)
         for (i,j) in zip(range(len(sms)), sms.spam)]
sms.index = index
sms.head(6)

INFO:nlpia.futil:Reading CSV with `read_csv(*('C:\\Anaconda\\python\\lib\\site-packages\\nlpia\\data\\sms-spam.csv',), **{'nrows': None, 'low_memory': False})`...


,spam,text
sms0,0,"Go until jurong point, crazy.. Available only in bu..."
sms1,0,Ok lar... Joking wif u oni...
sms2!,1,Free entry in 2 a wkly comp to win FA Cup final tkt...
sms3,0,U dun say so early hor... U c already then say...
sms4,0,"Nah I don't think he goes to usf, he lives around h..."
sms5!,1,FreeMsg Hey there darling it's been 3 week's now an...


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize
tfidf = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs = tfidf.fit_transform(raw_documents=sms.text).toarray()
len(tfidf.vocabulary_)

9232

In [30]:
tfidf_docs = pd.DataFrame(tfidf_docs)
tfidf_docs = tfidf_docs - tfidf_docs.mean()
tfidf_docs.shape

(4837, 9232)

In [31]:
sms.spam.sum()

638

In [32]:
from sklearn.decomposition import PCA
pca = PCA(n_components=16)
pca = pca.fit(tfidf_docs)
pca_topic_vectors = pca.transform(tfidf_docs)
columns = ['topic{}'.format(i) for i in range(pca.n_components)]
pca_topic_vectors = pd.DataFrame(pca_topic_vectors, columns=columns,index=index)
pca_topic_vectors.round(3).head(6)

,topic0,topic1,topic2,...,topic13,topic14,topic15
sms0,0.201,0.003,0.037,...,-0.032,-0.004,0.041
sms1,0.404,-0.094,-0.077,...,-0.028,0.046,-0.040
sms2!,-0.030,-0.048,0.090,...,-0.010,-0.039,0.062
sms3,0.329,-0.033,-0.035,...,-0.054,0.014,-0.074
sms4,0.002,0.031,0.038,...,0.030,-0.092,-0.025
sms5!,-0.016,0.059,0.014,...,0.083,0.012,0.001


In [33]:
tfidf.vocabulary_

{'go': 3807,
 'until': 8487,
 'jurong': 4675,
 'point': 6296,
 ',': 13,
 'crazy': 2549,
 '..': 21,
 'available': 1531,
 'only': 5910,
 'in': 4396,
 'bugis': 1973,
 'n': 5594,
 'great': 3894,
 'world': 8977,
 'la': 4811,
 'e': 3056,
 'buffet': 1971,
 '...': 25,
 'cine': 2277,
 'there': 8071,
 'got': 3855,
 'amore': 1296,
 'wat': 8736,
 'ok': 5874,
 'lar': 4848,
 'joking': 4642,
 'wif': 8875,
 'u': 8395,
 'oni': 5906,
 'free': 3604,
 'entry': 3195,
 '2': 471,
 'a': 1054,
 'wkly': 8933,
 'comp': 2386,
 'to': 8192,
 'win': 8890,
 'fa': 3328,
 'cup': 2608,
 'final': 3450,
 'tkts': 8180,
 '21st': 497,
 'may': 5272,
 '2005': 487,
 '.': 15,
 'text': 8020,
 '87121': 948,
 'receive': 6688,
 'question': 6574,
 '(': 9,
 'std': 7651,
 'txt': 8379,
 'rate': 6628,
 ')': 10,
 't': 7889,
 '&': 7,
 "c's": 2020,
 'apply': 1383,
 '08452810075': 115,
 'over': 6003,
 '18': 438,
 "'": 8,
 's': 6959,
 'dun': 3041,
 'say': 7034,
 'so': 7438,
 'early': 3069,
 'hor': 4207,
 'c': 2019,
 'already': 1268,
 'then': 

In [34]:
column_nums, terms = zip(*sorted(zip(tfidf.vocabulary_.values(),tfidf.vocabulary_.keys())))

In [35]:
terms

('!',
 '"',
 '#',
 '#150',
 '#5000',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '. .',
 '. . .',
 '. . . .',
 '. . . . .',
 '. ..',
 '..',
 '.. .',
 '.. . . .',
 '.. ... ...',
 '...',
 '... . . . .',
 '/',
 '0',
 '00',
 '00870405040',
 '0089',
 '01',
 '0121 2025050',
 '01223585236',
 '01223585334',
 '01256987',
 '02',
 '02/06',
 '02/09',
 '0207 153 9153',
 '0207 153 9996',
 '0207-083-6089',
 '02072069400',
 '02073162414',
 '02085076972',
 '03',
 '03530150',
 '04',
 '04/09',
 '05',
 '050703',
 '06',
 '06.05',
 '06/11',
 '07/11',
 '07008009200',
 '07046744435',
 '07090201529',
 '07090298926',
 '07099833605',
 '07123456789',
 '07732584351',
 '07734396839',
 '07742676969',
 '07753741225',
 '0776xxxxxxx',
 '07786200117',
 '077xxx',
 '078',
 '07801543489',
 '07808',
 '07808247860',
 '07808726822',
 '07815296484',
 '07821230901',
 '078498',
 '07880867867',
 '0789xxxxxxx',
 '07946746291',
 '0796xxxxxx',
 '07973788240',
 '07xxxxxxxxx',
 '08',
 '0800',
 '0800 0721072',
 '

In [36]:
weights = pd.DataFrame(pca.components_, columns=terms,
                       index=['topic{}'.format(i) for i in range(16)])
pd.options.display.max_columns = 8
weights.head(4).round(3)

,!,"""",#,#150,...,…,┾,〨ud,鈥
topic0,-0.071,0.008,-0.001,-0.000,...,-0.002,0.001,0.001,0.001
topic1,0.064,0.008,0.000,-0.000,...,0.003,0.001,0.001,0.001
topic2,0.071,0.027,0.000,0.001,...,0.002,-0.001,-0.001,-0.001
topic3,-0.059,-0.032,-0.001,-0.000,...,0.001,0.001,0.001,0.001


In [37]:
pd.options.display.max_columns = 12
deals = weights['! ;) :) half off free crazy deal only $ 80 %'.split()].round(3) * 100
deals

,!,;),:),half,off,free,crazy,deal,only,$,80,%
topic0,-7.1,0.1,-0.5,-0.0,-0.4,-2.0,-0.0,-0.1,-2.2,0.3,-0.0,-0.0
topic1,6.4,0.0,7.4,0.1,0.4,-2.3,-0.2,-0.1,-3.8,-0.1,-0.0,-0.2
topic2,7.1,0.2,-0.1,0.0,0.3,4.4,0.1,-0.1,0.7,0.0,0.0,0.1
topic3,-5.9,-0.3,-7.1,0.2,0.3,-0.2,0.0,0.1,-2.3,0.1,-0.1,-0.3
topic4,38.2,-0.1,-12.4,-0.1,-0.2,9.9,0.1,-0.2,3.0,0.3,0.1,-0.1
topic5,-26.5,0.1,-1.5,-0.3,-0.7,-1.4,-0.6,-0.2,-1.8,-0.9,0.0,0.0
topic6,-11.0,-0.5,19.8,-0.4,-0.9,-0.5,-0.2,-0.2,-1.4,-0.0,-0.0,-0.1
topic7,16.1,0.1,-17.7,0.7,0.8,-2.8,0.0,0.0,-1.8,-0.3,0.0,-0.1
topic8,34.6,0.1,5.0,-0.4,-0.5,0.0,-0.4,-0.4,3.3,-0.6,-0.0,-0.2
topic9,7.6,-0.3,16.6,1.5,-0.9,6.4,-0.5,-0.4,3.0,-0.5,-0.0,0.0


In [38]:
deals.T.sum()

topic0    -11.9
topic1      7.6
topic2     12.7
topic3    -15.5
topic4     38.5
topic5    -33.8
topic6      4.6
topic7     -5.0
topic8     40.5
topic9     32.5
topic10   -30.9
topic11   -43.6
topic12    22.4
topic13    49.9
topic14    17.9
topic15   -13.0
dtype: float64

In [70]:
tfidf_docs

array([[-2.56432277e-02, -5.84019981e-03, -2.28435274e-04, ...,
        -5.48526215e-05, -5.48526215e-05, -5.48526215e-05],
       [-2.56432277e-02, -5.84019981e-03, -2.28435274e-04, ...,
        -5.48526215e-05, -5.48526215e-05, -5.48526215e-05],
       [-2.56432277e-02, -5.84019981e-03, -2.28435274e-04, ...,
        -5.48526215e-05, -5.48526215e-05, -5.48526215e-05],
       ...,
       [-2.56432277e-02, -5.84019981e-03, -2.28435274e-04, ...,
        -5.48526215e-05, -5.48526215e-05, -5.48526215e-05],
       [-2.56432277e-02, -5.84019981e-03, -2.28435274e-04, ...,
        -5.48526215e-05, -5.48526215e-05, -5.48526215e-05],
       [-2.56432277e-02, -5.84019981e-03, -2.28435274e-04, ...,
        -5.48526215e-05, -5.48526215e-05, -5.48526215e-05]])

In [39]:
tfidf_docs.values

array([[-2.56432277e-02, -5.84019981e-03, -2.28435274e-04, ...,
        -5.48526215e-05, -5.48526215e-05, -5.48526215e-05],
       [-2.56432277e-02, -5.84019981e-03, -2.28435274e-04, ...,
        -5.48526215e-05, -5.48526215e-05, -5.48526215e-05],
       [-2.56432277e-02, -5.84019981e-03, -2.28435274e-04, ...,
        -5.48526215e-05, -5.48526215e-05, -5.48526215e-05],
       ...,
       [-2.56432277e-02, -5.84019981e-03, -2.28435274e-04, ...,
        -5.48526215e-05, -5.48526215e-05, -5.48526215e-05],
       [-2.56432277e-02, -5.84019981e-03, -2.28435274e-04, ...,
        -5.48526215e-05, -5.48526215e-05, -5.48526215e-05],
       [-2.56432277e-02, -5.84019981e-03, -2.28435274e-04, ...,
        -5.48526215e-05, -5.48526215e-05, -5.48526215e-05]])

In [40]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=16, n_iter=100)
svd_topic_vectors = svd.fit_transform(tfidf_docs.values)
svd_topic_vectors = pd.DataFrame(svd_topic_vectors, columns=columns,index=index)
svd_topic_vectors.round(3).head(6)

,topic0,topic1,topic2,topic3,topic4,topic5,...,topic10,topic11,topic12,topic13,topic14,topic15
sms0,0.201,0.003,0.037,0.011,-0.019,-0.053,...,0.007,-0.007,0.002,-0.036,-0.014,0.037
sms1,0.404,-0.094,-0.078,0.051,0.100,0.047,...,-0.004,0.036,0.043,-0.021,0.051,-0.042
sms2!,-0.030,-0.048,0.090,-0.067,0.091,-0.043,...,0.125,0.023,0.026,-0.020,-0.042,0.052
sms3,0.329,-0.033,-0.035,-0.016,0.052,0.056,...,0.022,0.023,0.073,-0.046,0.022,-0.070
sms4,0.002,0.031,0.038,0.034,-0.075,-0.093,...,0.028,-0.009,0.027,0.034,-0.083,-0.021
sms5!,-0.016,0.059,0.014,-0.006,0.122,-0.040,...,0.041,0.055,-0.037,0.075,-0.001,0.020


In [41]:
import numpy as np
svd_topic_vectors = (svd_topic_vectors.T / np.linalg.norm(svd_topic_vectors, axis=1)).T
svd_topic_vectors.iloc[:10].dot(svd_topic_vectors.iloc[:10].T).round(1)

,sms0,sms1,sms2!,sms3,sms4,sms5!,sms6,sms7,sms8!,sms9!
sms0,1.0,0.6,-0.1,0.6,-0.0,-0.3,-0.3,-0.1,-0.3,-0.3
sms1,0.6,1.0,-0.2,0.8,-0.2,0.0,-0.2,-0.2,-0.1,-0.1
sms2!,-0.1,-0.2,1.0,-0.2,0.1,0.4,0.0,0.3,0.5,0.4
sms3,0.6,0.8,-0.2,1.0,-0.2,-0.3,-0.1,-0.3,-0.2,-0.1
sms4,-0.0,-0.2,0.1,-0.2,1.0,0.2,0.0,0.1,-0.4,-0.2
sms5!,-0.3,0.0,0.4,-0.3,0.2,1.0,-0.1,0.1,0.3,0.4
sms6,-0.3,-0.2,0.0,-0.1,0.0,-0.1,1.0,0.1,-0.2,-0.2
sms7,-0.1,-0.2,0.3,-0.3,0.1,0.1,0.1,1.0,0.1,0.4
sms8!,-0.3,-0.1,0.5,-0.2,-0.4,0.3,-0.2,0.1,1.0,0.3
sms9!,-0.3,-0.1,0.4,-0.1,-0.2,0.4,-0.2,0.4,0.3,1.0


In [42]:
total_corpus_len = 0
for document_text in sms.text:
    total_corpus_len += len(casual_tokenize(document_text))
mean_document_len = total_corpus_len / len(sms)
round(mean_document_len, 2)

21.35

In [43]:
sum([len(casual_tokenize(t)) for t in sms.text]) * 1. / len(sms.text)

21.34794293983874

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
np.random.seed(42)
counter = CountVectorizer(tokenizer=casual_tokenize)
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=sms.text).toarray(), index=index)
column_nums, terms = zip(*sorted(zip(counter.vocabulary_.values(),counter.vocabulary_.keys())))
bow_docs.columns = terms

In [45]:
sms.loc['sms0'].text

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [46]:
bow_docs.loc['sms0'][bow_docs.loc['sms0'] > 0].head()

,            1
..           1
...          2
amore        1
available    1
Name: sms0, dtype: int64

In [47]:
from sklearn.decomposition import LatentDirichletAllocation as LDiA
ldia = LDiA(n_components=16, learning_method='batch')
ldia = ldia.fit(bow_docs)
ldia.components_.shape

(16, 9232)

In [48]:
pd.set_option('display.width', 75)
components = pd.DataFrame(ldia.components_.T, index=terms,columns=columns)
components.round(2).head(3)

,topic0,topic1,topic2,topic3,topic4,topic5,...,topic10,topic11,topic12,topic13,topic14,topic15
!,184.03,15.00,72.22,394.95,45.48,36.14,...,37.42,44.18,64.40,297.29,41.16,11.70
"""",0.68,4.22,2.41,0.06,152.35,0.06,...,8.42,11.42,0.07,62.72,12.27,0.06
#,0.06,0.06,0.06,0.06,0.06,2.07,...,0.06,0.06,1.07,4.05,0.06,0.06


In [49]:
components.topic3.sort_values(ascending=False)[:10]

!       394.952246
.       218.049724
to      119.533134
u       118.857546
call    111.948541
£       107.358914
,        96.954384
*        90.314783
your     90.215961
is       75.750037
Name: topic3, dtype: float64

In [50]:
ldia16_topic_vectors = ldia.transform(bow_docs)
ldia16_topic_vectors = pd.DataFrame(ldia16_topic_vectors,index=index, columns=columns)
ldia16_topic_vectors.round(2).head()

,topic0,topic1,topic2,topic3,topic4,topic5,...,topic10,topic11,topic12,topic13,topic14,topic15
sms0,0.00,0.62,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00
sms1,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.12,0.01,0.01,0.01,0.01
sms2!,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00
sms3,0.00,0.00,0.00,0.00,0.09,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00
sms4,0.39,0.00,0.33,0.00,0.00,0.00,...,0.00,0.00,0.09,0.00,0.00,0.00


In [51]:
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
X_train, X_test, y_train, y_test =train_test_split(ldia16_topic_vectors, sms.spam, test_size=0.5,random_state=271828)
lda = LDA(n_components=1)
lda = lda.fit(X_train, y_train)
sms['ldia16_spam'] = lda.predict(ldia16_topic_vectors)
round(float(lda.score(X_test, y_test)), 2)

C:\Anaconda\python\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


0.94

In [52]:
'''from itertools import product
all_pairs = [(word1, word2) for (word1, word2) in product(word_list,word_list) if not word1 == word2]'''

'from itertools import product\nall_pairs = [(word1, word2) for (word1, word2) in product(word_list,word_list) if not word1 == word2]'

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize
tfidf = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs = tfidf.fit_transform(raw_documents=sms.text).toarray()
tfidf_docs = tfidf_docs - tfidf_docs.mean(axis=0)
X_train, X_test, y_train, y_test = train_test_split(tfidf_docs,sms.spam.values, test_size=0.5, random_state=271828)
lda = LDA(n_components=1)
lda = lda.fit(X_train, y_train)
round(float(lda.score(X_train, y_train)), 3)

C:\Anaconda\python\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


1.0

In [54]:
round(float(lda.score(X_test, y_test)), 3)

0.748

In [55]:
ldia32 = LDiA(n_components=32, learning_method='batch')
ldia32 = ldia32.fit(bow_docs)
ldia32.components_.shape

(32, 9232)

In [56]:
ldia32_topic_vectors = ldia32.transform(bow_docs)
columns32 = ['topic{}'.format(i) for i in range(ldia32.n_components)]
ldia32_topic_vectors = pd.DataFrame(ldia32_topic_vectors, index=index,columns=columns32)
ldia32_topic_vectors.round(2).head()

,topic0,topic1,topic2,topic3,topic4,topic5,...,topic26,topic27,topic28,topic29,topic30,topic31
sms0,0.0,0.00,0.0,0.06,0.14,0.00,...,0.0,0.00,0.0,0.00,0.0,0.0
sms1,0.0,0.00,0.0,0.00,0.53,0.00,...,0.0,0.00,0.0,0.14,0.0,0.0
sms2!,0.0,0.00,0.0,0.00,0.00,0.65,...,0.0,0.00,0.0,0.00,0.0,0.0
sms3,0.0,0.11,0.0,0.00,0.39,0.00,...,0.0,0.00,0.0,0.00,0.0,0.0
sms4,0.0,0.00,0.0,0.00,0.00,0.00,...,0.0,0.47,0.0,0.00,0.0,0.0


In [57]:
X_train, X_test, y_train, y_test =train_test_split(ldia32_topic_vectors, sms.spam, test_size=0.5,random_state=271828)
lda = LDA(n_components=1)
lda = lda.fit(X_train, y_train)
sms['ldia32_spam'] = lda.predict(ldia32_topic_vectors)
X_train.shape

C:\Anaconda\python\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


(2418, 32)

In [58]:
round(float(lda.score(X_train, y_train)), 3)

0.933

In [59]:
round(float(lda.score(X_test, y_test)), 3)

0.936

In [60]:
'cityblock', 'cosine', 'euclidean', 'l1', 'l2', 'manhattan', 'braycurtis',
'canberra', 'chebyshev', 'correlation', 'dice', 'hamming', 'jaccard',
'kulsinski', 'mahalanobis', 'matching', 'minkowski', 'rogerstanimoto',
'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean',
'yule'

'yule'

In [61]:
lda = LDA(n_components=1)
lda = lda.fit(tfidf_docs, sms.spam)
sms['lda_spaminess'] = lda.predict(tfidf_docs)
((sms.spam - sms.lda_spaminess) ** 2.).sum() ** .5

C:\Anaconda\python\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


0.0

In [62]:
(sms.spam == sms.lda_spaminess).sum()

4837

In [63]:
len(sms)

4837

In [64]:
from sklearn.model_selection import cross_val_score
lda = LDA(n_components=1)
scores = cross_val_score(lda, tfidf_docs, sms.spam, cv=5)
"Accuracy: {:.2f} (+/-{:.2f})".format(scores.mean(), scores.std() * 2)

C:\Anaconda\python\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Anaconda\python\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Anaconda\python\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Anaconda\python\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Anaconda\python\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


'Accuracy: 0.76 (+/-0.03)'

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf_docs,sms.spam, test_size=0.33, random_state=271828)
lda = LDA(n_components=1)
lda.fit(X_train, y_train)

C:\Anaconda\python\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LinearDiscriminantAnalysis(n_components=1, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

In [66]:
lda.score(X_test, y_test).round(3)

0.765

In [67]:
X_train, X_test, y_train, y_test =train_test_split(pca_topic_vectors.values, sms.spam, test_size=0.3,random_state=271828)
lda = LDA(n_components=1)
lda.fit(X_train, y_train)

LinearDiscriminantAnalysis(n_components=1, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

In [68]:
lda.score(X_test, y_test).round(3)

0.962

In [69]:
lda = LDA(n_components=1)
scores = cross_val_score(lda, pca_topic_vectors, sms.spam, cv=10)
"Accuracy: {:.3f} (+/-{:.3f})".format(scores.mean(), scores.std() * 2)

'Accuracy: 0.957 (+/-0.021)'